In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
conditionDF = pd.read_csv('condition.csv')

In [2]:
conditionDF['visit_start_datetime']=pd.to_datetime(conditionDF['visit_start_datetime'])
conditionDF['visit_datetime_difference']=(pd.to_datetime(conditionDF['visit_end_datetime'])-pd.to_datetime(conditionDF['visit_start_datetime']))
conditionDF['condition_datetime_difference']=(pd.to_datetime(conditionDF['condition_end_datetime'])-pd.to_datetime(conditionDF['condition_start_datetime']))

In [3]:
persondrop=['location_id','provider_id','care_site_id','person_source_value','race_source_value',
            'gender_source_value','ethnicity_source_value','birth_datetime']
visitdrop=['visit_source_value','visit_start_date','visit_end_date','visit_end_datetime']
conditiondrop=['condition_source_value','condition_source_concept_id','condition_end_date',
              'condition_start_date','condition_end_datetime','condition_start_datetime']
conditionDF=conditionDF.drop(columns=persondrop)
conditionDF=conditionDF.drop(columns=visitdrop)
conditionDF=conditionDF.drop(columns=conditiondrop)

In [4]:
conditionDF.loc[pd.isnull(conditionDF['death_date']) , 'death'] = 0
conditionDF.loc[pd.isnull(conditionDF['death_date'])==False , 'death'] = 1
deathdrop=['death_date','death_datetime','death_type_concept_id','cause_source_concept_id']
conditionDF=conditionDF.drop(columns=deathdrop)

In [5]:
newconditionDF=conditionDF[conditionDF['visit_start_datetime'] == conditionDF.groupby('person_id')['visit_start_datetime'].transform('max')]

In [6]:
percent_missing = newconditionDF.isnull().sum() * 100 / len(conditionDF)
missing_value_df = pd.DataFrame({'column_name': conditionDF.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
person_id,person_id,0.000000
visit_occurrence_id,visit_occurrence_id,0.000000
year_of_birth,year_of_birth,0.000000
ethnicity_concept_id,ethnicity_concept_id,0.000000
month_of_birth,month_of_birth,0.000000
day_of_birth,day_of_birth,0.000000
race_concept_id,race_concept_id,4.159311
gender_concept_id,gender_concept_id,0.000000
visit_start_datetime,visit_start_datetime,0.000000
visit_concept_id,visit_concept_id,0.000000


In [7]:
newconditionDF = newconditionDF.apply(lambda x:x.fillna(x.value_counts().index[0]))

In [8]:
categorical= ['person_id','visit_occurrence_id','ethnicity_concept_id','race_concept_id','gender_concept_id',
             'visit_concept_id','visit_type_concept_id','condition_type_concept_id',
             'condition_concept_id','condition_occurrence_id','condition_status_concept_id']
for col in categorical:
    newconditionDF[col] = newconditionDF[col].astype('object')
newconditionDF['death'] = newconditionDF['death'].astype(int)

In [9]:
newconditionDF.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

person_id                       97916
visit_occurrence_id             94834
ethnicity_concept_id                2
race_concept_id                     6
gender_concept_id                   2
visit_concept_id                    3
visit_type_concept_id               1
condition_status_concept_id         1
condition_type_concept_id           2
condition_concept_id             6547
condition_occurrence_id        746263
dtype: int64

In [10]:
newconditionDF2=newconditionDF.drop(columns=['visit_start_datetime','visit_type_concept_id','condition_occurrence_id','condition_status_concept_id'])

In [18]:
columnstojoin=newconditionDF2[['person_id','visit_concept_id','condition_type_concept_id','death']]

In [22]:
final=columnstojoin.drop_duplicates().groupby('person_id').head(1)

In [13]:
newconditionDF3=newconditionDF2.groupby('person_id')['condition_concept_id'].agg({'count':len})

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [26]:
finalcondition=pd.merge(newconditionDF3, final, on='person_id')

In [27]:
finalcondition = finalcondition.rename(columns={'count': 'condition_num'})

In [29]:
finalcondition.to_csv("finalcondition.csv",index=False)